<a href="https://colab.research.google.com/github/DavidP0011/etl/blob/main/etl_Box_Office_Mojo_01raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalación de bibliotecas necesarias
# !pip install requests beautifulsoup4 pandas google-cloud-bigquery

import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.cloud import bigquery
import re
import time
import logging
from datetime import datetime

def scrape_and_upload_with_tconst(config):
    """
    Función para scrapear datos de Box Office Mojo y agregar el tconst desde IMDb Pro,
    luego subir los datos a Google BigQuery.

    Args:
        config (dict): Configuración de la función.
            - "project_id" (str): ID del proyecto en Google Cloud.
            - "dataset_id" (str): ID del conjunto de datos en BigQuery.
            - "table_id" (str): Nombre de la tabla en BigQuery.
            - "start_year" (int): Año inicial del rango.
            - "end_year" (int): Año final del rango.
            - "truncate_table" (bool): Si es True, truncar la tabla antes de subir los datos.
            - "log_file" (str): Ruta al archivo de log (opcional).
    """
    # Configuración del logger
    logger = logging.getLogger('BoxOfficeScraper')
    logger.setLevel(logging.DEBUG)  # Establece el nivel mínimo de log

    # Crear manejador para consola
    ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)  # Nivel de log para la consola

    # Crear manejador para archivo (si se especifica)
    if 'log_file' in config:
        fh = logging.FileHandler(config['log_file'])
        fh.setLevel(logging.DEBUG)  # Nivel de log para el archivo
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        fh.setFormatter(formatter)
        logger.addHandler(fh)

    # Formateador para la consola
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    ch.setFormatter(formatter)

    # Añadir manejadores al logger
    logger.addHandler(ch)

    # Configuración del cliente BigQuery
    client = bigquery.Client(project=config["project_id"])

    # Función interna para scrapeo de la tabla principal
    def scrape_box_office(year):
        url = f"https://www.boxofficemojo.com/year/world/{year}/"
        logger.info(f"Accediendo a la URL: {url}")
        try:
            response = requests.get(url)
            response.raise_for_status()
            logger.debug(f"Respuesta HTTP obtenida para el año {year}.")
        except requests.exceptions.RequestException as e:
            logger.error(f"Error al acceder a la página para el año {year}: {e}")
            return pd.DataFrame()

        soup = BeautifulSoup(response.content, "html.parser")

        # Buscar la tabla principal
        table = soup.find("table")
        if not table:
            logger.warning(f"No se encontró la tabla para el año {year}.")
            return pd.DataFrame()

        # Extraer encabezados para mapear columnas dinámicamente
        headers = [header.text.strip() for header in table.find_all("th")]
        logger.debug(f"Encabezados de la tabla para {year}: {headers}")

        # Mapeo de columnas basado en los encabezados actuales
        try:
            rank_idx = headers.index("Rank")
            title_idx = headers.index("Release Group")
            worldwide_gross_idx = headers.index("Worldwide")
            domestic_gross_idx = headers.index("Domestic")
            international_gross_idx = headers.index("Foreign")
        except ValueError as ve:
            logger.error(f"Error al encontrar índices de columnas: {ve}")
            return pd.DataFrame()

        rows = table.find_all("tr")[1:]  # Ignorar el encabezado

        # Procesar filas
        data = []
        total_rows = len(rows)
        logger.info(f"Procesando {total_rows} filas para el año {year}...")
        for idx, row in enumerate(rows, start=1):
            cols = row.find_all("td")
            if len(cols) < max(rank_idx, title_idx, worldwide_gross_idx, domestic_gross_idx, international_gross_idx) + 1:
                logger.debug(f"Fila {idx} ignorada por tener columnas insuficientes.")
                continue

            # Verificar si la primera columna es un número (Rank)
            rank_text = cols[rank_idx].text.strip()
            if not rank_text.isdigit():
                logger.debug(f"Fila {idx} ignorada porque 'Rank' no es un número: '{rank_text}'")
                continue  # Saltar filas donde el Rank no es un número

            try:
                rank = int(rank_text)
                title = cols[title_idx].text.strip()
                title_link = cols[title_idx].find("a")["href"]

                # Función interna para parsear valores de ganancias
                def parse_gross(text):
                    text = text.strip().replace("$", "").replace(",", "").replace("--", "").replace("-", "")
                    if text in ["", "--", "-"]:
                        return None
                    # Manejar posibles rangos o valores no numéricos
                    match = re.match(r'^\d+$', text)
                    return int(text) if match else None

                # Extraer y parsear las ganancias
                domestic_text = cols[domestic_gross_idx].text.strip()
                foreign_text = cols[international_gross_idx].text.strip()
                logger.debug(f"Fila {idx}: Domestic='{domestic_text}', Foreign='{foreign_text}'")

                worldwide_gross = parse_gross(cols[worldwide_gross_idx].text)
                domestic_gross = parse_gross(domestic_text)
                international_gross = parse_gross(foreign_text)

                # Agregar los datos al listado
                data.append({
                    "Year": year,
                    "Rank": rank,
                    "Title": title,
                    "Worldwide_Gross": worldwide_gross,
                    "Domestic_Gross": domestic_gross,
                    "International_Gross": international_gross,
                    "Detail_Link": f"https://www.boxofficemojo.com{title_link}"
                })
            except Exception as e:
                logger.error(f"Error procesando una fila en el índice {idx}: {e}")
                continue

            # Añadir retraso entre filas para evitar sobrecargar el servidor
            time.sleep(0.1)  # Esperar 100ms

        logger.info(f"Procesadas {len(data)} filas válidas para el año {year}.")
        return pd.DataFrame(data)

    # Función interna para obtener el tconst de IMDb Pro desde la página de detalles
    def get_tconst(detail_link):
        logger.debug(f"Obteniendo tconst desde: {detail_link}")
        try:
            response = requests.get(detail_link)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, "html.parser")

            # Buscar el enlace a IMDb Pro
            imdb_link = soup.find("a", href=lambda href: href and "pro.imdb.com/title" in href)
            if imdb_link:
                tconst_match = re.search(r'/title/(tt\d+)/', imdb_link["href"])
                if tconst_match:
                    tconst = tconst_match.group(1)
                    logger.debug(f"tconst encontrado: {tconst}")
                    return tconst
            logger.warning(f"tconst no encontrado en: {detail_link}")
        except requests.exceptions.RequestException as e:
            logger.error(f"Error al acceder al detalle: {detail_link} - {e}")
        except Exception as e:
            logger.error(f"Error al procesar el detalle: {detail_link} - {e}")
        return None

    # Scrapear datos para cada año en el rango
    all_data = pd.DataFrame()
    for year in range(config["start_year"], config["end_year"] + 1):
        logger.info(f"Scraping data for {year}...")
        year_data = scrape_box_office(year)

        if year_data.empty:
            logger.warning(f"No se encontraron datos para el año {year}.")
            continue

        # Agregar tconst a cada fila
        logger.info("Fetching IMDb tconst for each title...")
        year_data["tconst"] = year_data["Detail_Link"].apply(get_tconst)

        # Reportar cuántos tconst se encontraron
        tconst_found = year_data["tconst"].notnull().sum()
        tconst_total = len(year_data)
        logger.info(f"tconst encontrados: {tconst_found}/{tconst_total}")

        all_data = pd.concat([all_data, year_data], ignore_index=True)

    if all_data.empty:
        logger.error("No hay datos para subir a BigQuery.")
        return

    # Definir el esquema de BigQuery
    schema = [
        bigquery.SchemaField("Year", "INTEGER"),
        bigquery.SchemaField("Rank", "INTEGER"),
        bigquery.SchemaField("Title", "STRING"),
        bigquery.SchemaField("Worldwide_Gross", "INTEGER"),
        bigquery.SchemaField("Domestic_Gross", "INTEGER"),
        bigquery.SchemaField("International_Gross", "INTEGER"),
        bigquery.SchemaField("Detail_Link", "STRING"),
        bigquery.SchemaField("tconst", "STRING"),
    ]

    # Convertir las columnas de ganancias a tipo integer (ya se hizo en parse_gross)
    # Verificar tipos de datos
    expected_types = {
        "Year": int,
        "Rank": int,
        "Title": str,
        "Worldwide_Gross": pd.Int64Dtype(),
        "Domestic_Gross": pd.Int64Dtype(),
        "International_Gross": pd.Int64Dtype(),
        "Detail_Link": str,
        "tconst": str,
    }

    for column, dtype in expected_types.items():
        if column in all_data.columns:
            all_data[column] = all_data[column].astype(dtype)
        else:
            all_data[column] = None  # Agregar columna si falta

    # Registro de datos para depuración
    logger.info("Vista previa de los datos recopilados:")
    logger.debug(all_data.head())

    logger.info("Resumen de valores nulos:")
    logger.debug(all_data.isnull().sum())

    # Subir datos a BigQuery con esquema definido
    table_id = f"{config['project_id']}.{config['dataset_id']}.{config['table_id']}"
    logger.info(f"Uploading data to BigQuery table: {table_id}...")

    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE if config.get("truncate_table", False) else bigquery.WriteDisposition.WRITE_APPEND,
        source_format=bigquery.SourceFormat.PARQUET,
    )

    try:
        job = client.load_table_from_dataframe(all_data, table_id, job_config=job_config)
        job.result()  # Esperar a que termine el trabajo
        logger.info("Datos subidos exitosamente!")
        logger.info(f"Total de registros cargados: {len(all_data)}")
    except Exception as e:
        logger.error(f"Error al subir datos a BigQuery: {e}")



In [3]:

# Configuración de ejemplo
config = {
    "project_id": "animum-dev-datawarehouse",   # Cambia esto por tu ID de proyecto
    "dataset_id": "BOMojo_staging_01",   # Cambia esto por tu ID de dataset
    "table_id": "WorldwideBoxOffice_01",         # Cambia esto por el nombre de la tabla
    "start_year": 2000,                # Año inicial del rango
    "end_year": 2024,                   # Año final del rango
    "delete_table": True               # True para borrar los registros la tabla antes de cargar los datos
}

# Ejecutar función
scrape_and_upload_with_tconst(config)

2025-02-10 09:01:14,615 - INFO - Scraping data for 2000...
2025-02-10 09:01:14,615 - INFO - Scraping data for 2000...
INFO:BoxOfficeScraper:Scraping data for 2000...
2025-02-10 09:01:14,622 - INFO - Accediendo a la URL: https://www.boxofficemojo.com/year/world/2000/
2025-02-10 09:01:14,622 - INFO - Accediendo a la URL: https://www.boxofficemojo.com/year/world/2000/
INFO:BoxOfficeScraper:Accediendo a la URL: https://www.boxofficemojo.com/year/world/2000/
DEBUG:BoxOfficeScraper:Respuesta HTTP obtenida para el año 2000.
DEBUG:BoxOfficeScraper:Encabezados de la tabla para 2000: ['Rank', 'Release Group', 'Worldwide', 'Domestic', '%', 'Foreign', '%']
2025-02-10 09:01:16,223 - INFO - Procesando 200 filas para el año 2000...
2025-02-10 09:01:16,223 - INFO - Procesando 200 filas para el año 2000...
INFO:BoxOfficeScraper:Procesando 200 filas para el año 2000...
DEBUG:BoxOfficeScraper:Fila 1: Domestic='$215,409,889', Foreign='$330,978,219'
DEBUG:BoxOfficeScraper:Fila 2: Domestic='$187,705,427', F

KeyboardInterrupt: 